In [1]:
import os

In [2]:
%pwd

'/Users/sushraireddy/Desktop/Kidney-Disease-Classfication/Kidney-Disease-Classfication/research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path  # Correct import

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir:Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

[2025-05-30 13:46:39,088: INFO: __init__: Welcome to custom log]


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,  # match YAML key
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from cnnClassifier import logger

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file

            os.makedirs("artifacts/data_ingestion", exist_ok=True)

            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            download_url = prefix + file_id

            gdown.download(download_url, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            return zip_download_dir

        except Exception as e:
            logger.error(f"Failed to download file due to: {e}")
            raise

    def extract_zip_file(self) -> None:
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

        except Exception as e:
            logger.error(f"Failed to extract zip file due to: {e}")
            raise

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise

[2025-05-30 13:46:50,034: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-30 13:46:50,036: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-30 13:46:50,037: INFO: common: created directory at: artifacts]
[2025-05-30 13:46:50,038: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-30 13:46:50,039: INFO: 3469047994: Downloading data from https://drive.google.com/file/d/1wqlI7hsen8Q7wpjzs3pUeGlDZKqci4W-/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1wqlI7hsen8Q7wpjzs3pUeGlDZKqci4W-
From (redirected): https://drive.google.com/uc?export=download&id=1wqlI7hsen8Q7wpjzs3pUeGlDZKqci4W-&confirm=t&uuid=e0d6c72a-6879-492f-98f6-dd1be7bc0954
To: /Users/sushraireddy/Desktop/Kidney-Disease-Classfication/Kidney-Disease-Classfication/artifacts/data_ingestion/data.zip
100%|██████████| 943M/943M [04:16<00:00, 3.67MB/s] 

[2025-05-30 13:51:12,204: INFO: 3469047994: Downloaded data from https://drive.google.com/file/d/1wqlI7hsen8Q7wpjzs3pUeGlDZKqci4W-/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [10]:
from cnnClassifier.utils.common import read_yaml
from pathlib import Path

config = read_yaml(Path("config/config.yaml"))
print(config)

[2025-05-30 12:53:57,183: INFO: common: yaml file: config/config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1wqlI7hsen8Q7wpjzs3pUeGlDZKqci4W-/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
